##### Introduction as in Apache Website

In [1]:
from pyspark import S

sc = SparkContext(appName="Test")

##### Reading Text File

In [3]:
textFile = sc.textFile(r"file:///C:\Users\Dell\Desktop\Hadoop\Spark\bootStrap.sh")

In [4]:
textFile.count()

27

In [5]:
textFile.collect()

['\twget --quiet https://repo.anaconda.com/archive/Anaconda3-2019.03-Linux-x86_64.sh -O $HOME/conda.sh',
 '\tchmod 777 $HOME/conda.sh',
 '\t/bin/bash /$HOME/conda.sh -b -p $HOME/conda',
 '\tchmod -R 777 $HOME/conda',
 "\techo -e '\\nexport PATH=$HOME/conda/bin:$PATH' >> $HOME/.bashrc && source $HOME/.bashrc",
 '\tconda install -y pyarrow pyspark',
 '',
 "#sudo echo -e '\\nexport PYSPARK_PYTHON=/home/hadoop/conda/bin/python' >> /etc/spark/conf/spark-env.sh",
 '#sudo sed -i "s/## region=us-east-1/region=us-east-2/" /etc/hue/conf.empty/hue.ini',
 '',
 '',
 'sudo pyspark --packages org.apache.spark:spark-avro:2.4.0',
 '',
 'This failed to launch !!!!',
 '',
 '',
 '* How to know the large_temp_2.csv FOLDER size?',
 '    * **Ans**: hadoop fs -du -s -h /user/livy/large_temp_2.csv',
 '\t\t* 34.2 M  /user/livy/large_temp_2.csv',
 '',
 '* can 2 jobs, one for append and one for count work in parallel ? ',
 '* what is the read and count for csv file?',
 '* what is the read and count for parquete f

##### Filter == Query in pandas

In [7]:
textFile[textFile.value.contains("abc")].count()

AttributeError: 'RDD' object has no attribute 'value'

###### Note that the above case has created RDD and dataframe because the file has been read using spark context and not SQL context

Let us create sparkSession

In [19]:
from pyspark import SQLContext
spark = SQLContext(sc).sparkSession

In [39]:
text = spark.read.text(r"file:///C:\Users\Dell\Desktop\Hadoop\Spark\bootStrap.sh")

In [40]:
text[text.value.contains("get")].collect()

[Row(value='\twget --quiet https://repo.anaconda.com/archive/Anaconda3-2019.03-Linux-x86_64.sh -O $HOME/conda.sh')]

In [41]:
from pyspark.sql.functions import *

##### Creating new column in spark dataframe

In [43]:
text1 = text.select(size(split(text.value,"\s+")).name("wordCount"))
text1.show()

+---------+
|wordCount|
+---------+
|        6|
|        4|
|        6|
|        5|
|       10|
|        6|
|        1|
|        7|
|        6|
|        1|
|        1|
|        4|
|        1|
|        5|
|        1|
|        1|
|        8|
|        9|
|        5|
|        1|
+---------+
only showing top 20 rows



In [55]:
text.select(size(split(text.value,"\s+")).name("wordCount")).agg(max(col("wordCount"))).show()

+--------------+
|max(wordCount)|
+--------------+
|            25|
+--------------+



##### Simulating MapReduce action

In [59]:
text.select(split(text.value,"\s+").name("word")).collect()

[Row(word=['', 'wget', '--quiet', 'https://repo.anaconda.com/archive/Anaconda3-2019.03-Linux-x86_64.sh', '-O', '$HOME/conda.sh']),
 Row(word=['', 'chmod', '777', '$HOME/conda.sh']),
 Row(word=['', '/bin/bash', '/$HOME/conda.sh', '-b', '-p', '$HOME/conda']),
 Row(word=['', 'chmod', '-R', '777', '$HOME/conda']),
 Row(word=['', 'echo', '-e', "'\\nexport", "PATH=$HOME/conda/bin:$PATH'", '>>', '$HOME/.bashrc', '&&', 'source', '$HOME/.bashrc']),
 Row(word=['', 'conda', 'install', '-y', 'pyarrow', 'pyspark']),
 Row(word=['']),
 Row(word=['#sudo', 'echo', '-e', "'\\nexport", "PYSPARK_PYTHON=/home/hadoop/conda/bin/python'", '>>', '/etc/spark/conf/spark-env.sh']),
 Row(word=['#sudo', 'sed', '-i', '"s/##', 'region=us-east-1/region=us-east-2/"', '/etc/hue/conf.empty/hue.ini']),
 Row(word=['']),
 Row(word=['']),
 Row(word=['sudo', 'pyspark', '--packages', 'org.apache.spark:spark-avro:2.4.0']),
 Row(word=['']),
 Row(word=['This', 'failed', 'to', 'launch', '!!!!']),
 Row(word=['']),
 Row(word=['']),


In [64]:
 text.select(split(text.value,"\s+").name("word")).groupby("word").count().show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|[, *, if, yes,, c...|    1|
|[#sudo, sed, -i, ...|    1|
|[, *, obviously, ...|    1|
|[#sudo, echo, -e,...|    1|
|[, *, **Ans**:, h...|    1|
|[*, what, is, the...|    1|
|[, /bin/bash, /$H...|    1|
|[, echo, -e, '\ne...|    1|
|[sudo, pyspark, -...|    1|
|[*, How, to, know...|    1|
|[, wget, --quiet,...|    1|
|[, chmod, 777, $H...|    1|
|[This, failed, to...|    1|
|[, chmod, -R, 777...|    1|
|[*, what, is, the...|    1|
|[, *, if, no,, ca...|    1|
|[*, can, panda, h...|    1|
|[*, can, 2, jobs,...|    1|
|[, *, 34.2, M, /u...|    1|
|                  []|    7|
+--------------------+-----+
only showing top 20 rows



##### Explode, GroupBy and OrderBy

In [79]:
 df = text.select(explode(split(text.value,"\S +")).name("word")).groupby("word").count().orderBy(desc("count"))

In [81]:
%%timeit
df.show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    |   17|
|                   -|   10|
|                  th|    8|
|                   i|    6|
|                  an|    5|
|                  fo|    4|
|                  ca|    4|
|                   o|    3|
|                    |    3|
|                spar|    3|
|                coun|    3|
|               	chmo|    2|
|                 wha|    2|
|            '\nexpor|    2|
|/user/livy/large_...|    2|
|                  on|    2|
|                  no|    2|
|                pand|    2|
|                  cs|    2|
|      $HOME/conda.sh|    2|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    |   17|
|                   -|   10|
|                  th|    8|
|                   i|    6|
|                  an|    5|
|                  ca|    4|
|                

In [82]:
df.show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    |   17|
|                   -|   10|
|                  th|    8|
|                   i|    6|
|                  an|    5|
|                  fo|    4|
|                  ca|    4|
|                   o|    3|
|                    |    3|
|                coun|    3|
|                spar|    3|
|               	chmo|    2|
|/user/livy/large_...|    2|
|                  no|    2|
|                  cs|    2|
|                 wha|    2|
|                pand|    2|
|                  77|    2|
|      $HOME/conda.sh|    2|
|                  on|    2|
+--------------------+-----+
only showing top 20 rows



##### Importance of Cache

In [83]:
df.cache()

DataFrame[word: string, count: bigint]

In [87]:
%%timeit
df.show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    |   17|
|                   -|   10|
|                  th|    8|
|                   i|    6|
|                  an|    5|
|                  ca|    4|
|                  fo|    4|
|                    |    3|
|                coun|    3|
|                   o|    3|
|                spar|    3|
|                 wha|    2|
|               	chmo|    2|
|                  cs|    2|
|                  77|    2|
|/user/livy/large_...|    2|
|                  on|    2|
|      $HOME/conda.sh|    2|
|                  no|    2|
|            '\nexpor|    2|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    |   17|
|                   -|   10|
|                  th|    8|
|                   i|    6|
|                  an|    5|
|                  ca|    4|
|                

##### One more way of creating spark session

In [93]:
from pyspark.sql import SparkSession
spark2 = SparkSession.builder.appName("temp").getOrCreate()

In [95]:
spark2.read.text(r"file:///C:\Users\Dell\Desktop\Hadoop\Spark\bootStrap.sh").show()

+--------------------+
|               value|
+--------------------+
|	wget --quiet htt...|
|	chmod 777 $HOME/...|
|	/bin/bash /$HOME...|
|	chmod -R 777 $HO...|
|	echo -e '\nexpor...|
|	conda install -y...|
|                    |
|#sudo echo -e '\n...|
|#sudo sed -i "s/#...|
|                    |
|                    |
|sudo pyspark --pa...|
|                    |
|This failed to la...|
|                    |
|                    |
|* How to know the...|
|    * **Ans**: ha...|
|		* 34.2 M  /user...|
|                    |
+--------------------+
only showing top 20 rows



##### Quick Note on Cluster and spark program execution on them
* one single python spark program has 2 logical components.
    * Driver tasks
        * To assgin a task and collect the task operation output.
        * It cordianates with resource managers like yarn and mesos for resource availing. This is transperant to the programmer as he does only read, write and dataframe related operations. 
    * Worker tasks
        * indicates its presence to resource allocator and hence gets nominated for a task
        * executes the instructions as per driver program
        * note that here operation is limited to the data available in its scope and hence execution is faster and also parallel processing is feasible.
        
        
* Monitoring can be done through: http://<driver-node>:4040
* There are application like livy which also provides interactive access to jobs and thier features.

##### Job Scheduling
* unlike, in any other scheduling context, here we are talking about the resource scheduling


* Static Partitioning:
    * number of resources to a job can be limited through spark submit arguments
        * --num-executors : number of executors for a job
        * --executor-memory
        * --executor-cores : Note cores are logical units of computation in CPU



* Dynamic Resource Allocation:
    * Jobs can return the resource after current usage and request again for later use.
    * set spark.dynamicAllocation.enabled to true
    * set spark.shuffle.service.enabled to true
        * enabling resource executor to be removed but not the shuffle files. So intermediate results are retained so that when executor requires it, can be fetched. 
        * Shuffle service which always run and can collect all the shuffle files info across application can avoid a scenario of new executor trying to access the old executor shuffule file in progress writing.
            * Here Old executor can submit a shuffle right to the shuffle services and terminate gracefully.
            * Shuffle service will handle the situation of new executor requesting the old shuffle file content.

    * spark.dynamicAllocation.schedulerBacklogTimeout is used to trigger the request.
        * if not availed per first request, it will request again for every spark.dynamicAllocation.sustainedSchedulerBacklogTimeout seconds. 
        * executor count is increased exponentially for every subsequent request
            * this is because job was waiting for it for so long and has to catch up for the waiting period
            * it also acts as a buffer for future use (as the intial request did not get fulfill).
            
    * executors are removed after spark.dynamicAllocation.executorIdleTimeout seconds.
            
        
        
* Scheduling Within an Application:
    * Fair scheduling: all jobs within application like save, collect etc are by default executed FIFO manner but Fair scheduling can allocate resources to smaller jobs even wen long jobs are being execute.
        * conf.set("spark.scheduler.mode", "FAIR")
        * Pools can also have 
            * weights to decide which has to be given higher preference
            * minShare to decide which is the minimum share despite being lower weights
        